# Accessing the SCP York database
A quick demonstration on how to do this using the R psotgres library
`install.packages('RPostgreSQL')`

For questions contact daniel.ellis@york.ac.[uk]


R can be installed on anaconda using `conda install -c r r-irkernel`

In [1]:
library('RPostgreSQL')

Loading required package: DBI


In [2]:
# Lets log into the database

pg = dbDriver("PostgreSQL")

cred = scan('~/.postgres_login', what="character")

con = dbConnect(pg, user= cred[1], password= cred[2],
                host= paste(cred[3],".york.ac.uk",sep=''),
                port=5432, dbname=cred[4])



### Now we are connected, we can have a look at querying.


## Reading a Table
Lets read the first 5 results from the countries table.

Double quotation marks are not usually needed.


In [3]:
sql = 'SELECT * from "comm_trade.countries" LIMIT 5;'

In [4]:
# read back the full table
dtab = dbGetQuery(con, sql)

In [5]:
dtab

index,area_code,name,iso
0,10,Antarctica,ATA
1,100,Bulgaria,BGR
2,104,Myanmar,MMR
3,108,Burundi,BDI
4,112,Belarus,BLR


### If we wish to only take the 'name' and 'area_code' columns:

In [6]:
sql = '
        SELECT name,area_code 
        from "comm_trade.countries" 
        LIMIT 5;
'
# read back the full table
dtab = dbGetQuery(con, sql)
dtab

name,area_code
Antarctica,10
Bulgaria,100
Myanmar,104
Burundi,108
Belarus,112


## Structure
Databases consist of groups of tables (Schemas). These represent the different projects that may exist

# List all schemas
Use `\dn+` in psql

In [7]:
sql = 'SELECT schema_name
       FROM information_schema.schemata'
dbGetQuery(con, sql)


schema_name
pg_catalog
information_schema
public
scpdata
comm_trade
fao


### If we then wish to see what tables exist within a certain schema (eg. fao)

In [8]:
sql = "
        SELECT table_name 
        FROM information_schema.tables
        WHERE table_schema='fao';
"
dtab = dbGetQuery(con, sql)
dtab


table_name
element
flags
countries
items
cb_production


### Finally we may also view the properties of a table within that schema, e.g. the commodity balance table from the fao schema

Use `\dt` tablename in psql

In [9]:
sql = "
    SELECT table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,character_maximum_length
    FROM information_schema.columns 
    WHERE table_schema = 'fao' AND table_name = 'cb_production'
;"
dtab = dbGetQuery(con, sql)
dtab


table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,character_maximum_length
scpdata,fao,cb_production,area_code,1,NO,integer,NA
scpdata,fao,cb_production,item_code,2,NO,integer,NA
scpdata,fao,cb_production,year,3,NO,integer,NA
scpdata,fao,cb_production,unit,4,YES,character varying,500
scpdata,fao,cb_production,value,5,NO,numeric,NA
scpdata,fao,cb_production,flag,6,YES,character varying,5


# Querying 
### Now lets explore the contents
(showing only the first 5 results)

In [10]:
sql = 'SELECT * from fao.cb_production LIMIT 5;'
dtab = dbGetQuery(con, sql)
dtab

area_code,item_code,year,unit,value,flag
2,2617,1961,tonnes,15100,S
2,2617,1962,tonnes,15100,S
2,2617,1963,tonnes,15100,S
2,2617,1964,tonnes,18400,S
2,2617,1965,tonnes,20400,S


### What if we want only the values for year 2013

In [11]:
sql = 'SELECT * from fao.cb_production WHERE year = 2013 LIMIT 5;'
dbGetQuery(con, sql)

area_code,item_code,year,unit,value,flag
2,2617,2013,tonnes,78597,S
2,2513,2013,tonnes,514000,S
2,2600,2013,tonnes,441732,S
2,2614,2013,tonnes,3000,S
2,2661,2013,tonnes,13917,S


## Asigning an alias in the query
We may wish to shorten the name of what we are querying to save typing later on. Here we give the commodity balance production table the name `p`.

In [12]:
sql = '
    SELECT * 
    FROM fao.cb_production p 
    WHERE p.year = 2013 
    LIMIT 5;'
dbGetQuery(con, sql)

area_code,item_code,year,unit,value,flag
2,2617,2013,tonnes,78597,S
2,2513,2013,tonnes,514000,S
2,2600,2013,tonnes,441732,S
2,2614,2013,tonnes,3000,S
2,2661,2013,tonnes,13917,S


## Link two tables
Sometimes we wish to combine two tables to produce an output. We can do this through one of the many possible join commands. Here we join our item table with the current one on the item_code columns (conveniently named the same in both tables) using `JOIN items i ON(p.item_code = i.item_code)`


In [13]:
sql = '
    SELECT * 
    FROM fao.cb_production p 
    JOIN fao.items i ON(p.item_code = i.item_code) 
    WHERE p.year=2013  
    LIMIT 10;'
dbGetQuery(con, sql)

area_code,item_code,year,unit,value,flag,item_code,item
2,2617,2013,tonnes,78597,S,2617,Apples and products
2,2513,2013,tonnes,514000,S,2513,Barley and products
2,2600,2013,tonnes,441732,S,2600,Brans
2,2614,2013,tonnes,3000,S,2614,"Citrus, Other"
2,2661,2013,tonnes,13917,S,2661,Cotton lint
2,2559,2013,tonnes,27412,S,2559,Cottonseed
2,2594,2013,tonnes,11769,S,2594,Cottonseed Cake
2,2575,2013,tonnes,4707,S,2575,Cottonseed Oil
2,2625,2013,tonnes,303788,S,2625,"Fruits, Other"
2,2620,2013,tonnes,610570,S,2620,Grapes and products (excl wine)


## Unique (DISTINCT) values
If we dont want duplicates, we can always call upon the distinct selector. 

In [14]:
# Get all the years in the year column, and return a list with no duplicates
sql = 'SELECT DISTINCT year 
       FROM fao.cb_production 
       ORDER BY "year" DESC
       LIMIT 5
'
dbGetQuery(con, sql)



year
2013
2012
2011
2010
2009
